In [ ]:
import stanza    
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

In [1]:
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('../datasets/2018-EI-oc-En-fear-dev.txt', sep="\t", header=None, skiprows=1)

'''
li = []
all_files= {'../datasets/2018-EI-oc-En-joy-dev.txt', '../datasets/2018-EI-oc-En-sadness-dev.txt', '../datasets/2018-EI-oc-En-anger-dev.txt', '../datasets/2018-EI-oc-En-fear-dev.txt'}

for filename in all_files:
    df = pd.read_csv(filename, sep="\t", header=None, skiprows=1)
    li.append(df)

dataset = pd.concat(li, axis=0, ignore_index=True)
'''
dataset.columns = ['date', 'text', 'emotion', 'level']
dataset= dataset.sample(frac=1)


In [ ]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import emoji
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

corpus = []
ps = PorterStemmer()

for index, row in dataset.iterrows():
    tweet = row['text']

    #handle users
    tweet = re.sub('@.*', '@user', tweet) 

    
    
    tweet = tweet.lower().split()
    #tweet = nltk.word_tokenize(tweet)

    # stemming and stop word removal
    tweet = ' '.join([lemmatizer.lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))])
    

    #tweet = nlp(tweet) # run annotation over a sentence
    
    
    #emojis
    tweet = emoji.demojize(tweet)
    
    corpus.append(tweet)

print(corpus)


In [2]:
from bert_embedding import BertEmbedding
import re
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

corpus = []

for index, row in dataset.iterrows():
    tweet = row['text']
    corpus.append(tweet)

bert = BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')
results = bert(corpus)
print(results)

y([-0.5495291 , -0.7918779 ,  0.42133626, ..., -0.45186043,
        1.2712893 , -0.30512768], dtype=float32), array([ 0.19070424, -0.03193848,  0.38265812, ...,  0.36015743,
        0.31004107,  0.67703635], dtype=float32), array([-0.5449671 ,  0.31310982, -1.1478119 , ...,  0.19127414,
       -0.04789663,  0.29773694], dtype=float32), array([-0.39347076,  0.19082141,  0.23770455, ...,  0.26232812,
       -0.40102518,  0.41818148], dtype=float32), array([-0.0806164 ,  0.1232103 , -0.6826154 , ...,  0.30468202,
       -0.07247794,  0.22103599], dtype=float32), array([-0.41422665,  0.13156877,  0.06671315, ...,  0.23848504,
        0.31772968, -0.38602728], dtype=float32), array([-0.37396076,  0.11147672,  0.68977755, ...,  0.13982588,
       -0.6207702 , -0.03338522], dtype=float32), array([ 0.7093064 ,  0.7645921 , -0.36428154, ...,  0.51205176,
        0.09070683, -0.2818346 ], dtype=float32), array([ 1.057663  ,  1.3295591 , -0.7451169 , ...,  0.49303836,
        0.10944813, -0.74136

In [3]:
import numpy as np
averaged = []
for sent in results:
    averaged.append(np.mean(sent[1], axis = 0, dtype=np.float64))

corpus=averaged
X=np.array(corpus)

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(corpus)

print(tfidf_vectorizer_vectors)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


vectorizer = CountVectorizer(max_features = 1500)
#X = vectorizer.fit_transform(corpus).toarray()
#X = tfidf_vectorizer_vectors.toarray()
y = []
for index, row in dataset.iterrows():
    y.append(int(row['level'][0]))
    '''
    if(int(row['level'][0])==3):
        y.append(1) 
    elif(int(row['level'][0])>0):
        y.append(1) 
    else:
        y.append(0)
    '''
    
    
y = np.array(y)
#print(vectorizer.get_feature_names())
#print(type(X[0]), y.shape)

In [ ]:
'''
from nltk.featstruct import FeatStruct

X = []
i=0

for index, row in dataset.iterrows():
    element= []
    element.append(tfidf_vectorizer_vectors[i].toarray())
    element.append(np.array([0 if i !=len(row['text']) else 1 for i in range(500)]))
    element=np.array(element)
    element=element.flatten()
    X.append(element)
    i+=1
    fs1 = FeatStruct(tdidf=tfidf_vectorizer_vectors[i], size=[0 if i !=len(row['text']) else 1 for i in range(500)])
    X.append(fs1)
X=np.array(X)
print(X.shape)
'''

In [5]:
# Split dataset into training and test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(311, 1024) (311,)
(78, 1024) (78,)


In [6]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[31  3 11  2]
 [ 8  1  4  0]
 [ 6  2  2  0]
 [ 2  1  5  0]]
Accuracy:  0.4358974358974359
Precision:  0.43290043290043284
Recall:  0.4358974358974359
F1:  0.4301282051282051


In [7]:
# SVM

from sklearn.svm import SVC

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[47  0  0  0]
 [13  0  0  0]
 [10  0  0  0]
 [ 8  0  0  0]]
Accuracy:  0.6025641025641025
Precision:  0.36308349769888226
Recall:  0.6025641025641025
F1:  0.45312820512820506
/home/joana/envs/python37/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[40  4  3  0]
 [ 8  2  3  0]
 [ 7  1  2  0]
 [ 5  1  2  0]]
Accuracy:  0.5641025641025641
Precision:  0.46901709401709396
Recall:  0.5641025641025641
F1:  0.5079022742574144
/home/joana/envs/python37/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# SGDC classifier

from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[39  6  2  0]
 [11  2  0  0]
 [ 8  1  1  0]
 [ 6  1  1  0]]
Accuracy:  0.5384615384615384
Precision:  0.43257211538461543
Recall:  0.5384615384615384
F1:  0.4707239489848186
/home/joana/envs/python37/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [9]:
# Perceptron
from sklearn.linear_model import Perceptron

classifier = Perceptron() 
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred)) 
print('Accuracy: ', accuracy_score(y_test, y_pred)) 
print('Precision: ', precision_score(y_test, y_pred, average='weighted')) 
print('Recall: ', recall_score(y_test, y_pred, average='weighted')) 
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[39  6  2  0]
 [ 8  3  2  0]
 [ 7  0  3  0]
 [ 5  1  2  0]]
Accuracy:  0.5769230769230769
Precision:  0.4910401274808055
Recall:  0.5769230769230769
F1:  0.5273603172441736
/home/joana/envs/python37/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[23 10 13  1]
 [ 9  4  0  0]
 [ 7  2  1  0]
 [ 4  3  1  0]]
Accuracy:  0.358974358974359
Precision:  0.3659364571237276
Recall:  0.358974358974359
F1:  0.35990028490028486


In [11]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

[[47  0  0  0]
 [13  0  0  0]
 [10  0  0  0]
 [ 8  0  0  0]]
Accuracy:  0.6025641025641025
Precision:  0.36308349769888226
Recall:  0.6025641025641025
F1:  0.45312820512820506
/home/joana/envs/python37/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Simple test

import os
import numpy as np

tweet = input("Enter tweet: ")
tweet = re.sub('[^a-zA-Z]', ' ', rev).split()
tweet = ' '.join([ps.stem(w) for w in tweet])
X = vectorizer.transform([tweet]).toarray()

print(X.shape)
print(X)

print("Sentiment level: ", classifier.predict(X))